In [ ]:
import pandas as pd
import numpy as np
import math

ALPHA = 0.8
eps = 1e-4
data = pd.read_csv("/Users/hasan.nayir/Projects/Payten/app2scale_reinforcement_learning/server_client_v4/data.csv")

In [ ]:
# Reward

ALPHA = 0.8

data["utilization"] = np.minimum(data["cpu_usage"]/(data["cpu"]/10),1)
data["performance_request"] = np.minimum(round(data['num_request'] /  (data['expected_tps']),6),1)

for i in range(0, data.shape[0]):
    temp = data.iloc[i,:]
    if temp["response_time"] >= 20:
        data.loc[i, "performance_response"] = 20/temp["response_time"]
    else:
        data.loc[i, "performance_response"] = 1

data["performance"] = 0.5*data["performance_request"] + 0.5*data["performance_response"]
data["reward"] = data["performance"] * ALPHA + data["utilization"] * (1-ALPHA)


drop_rows = (data["cpu_usage"] != 0) | (data["memory_usage"] != 0)
hybrid_data = data[drop_rows].reset_index(drop=True)

In [ ]:
import gymnasium as gym
import numpy as np
import os
import pandas as pd
import ray._private.utils
from gymnasium.spaces import Discrete, Box
from ray.rllib.models.preprocessors import get_preprocessor
from ray.rllib.evaluation.sample_batch_builder import SampleBatchBuilder
from ray.rllib.offline.json_writer import JsonWriter
from itertools import product


def convert_data_to_batch(df, writer, eps_id_list):
    number_episodes = int(df.shape[0]/episode_length)
    remained_steps = df.shape[0]-number_episodes*episode_length
    for eps_id,idx in zip(eps_id_list, range(len(eps_id_list))):
        print(eps_id)
        try:
            data = df.iloc[idx*episode_length:(idx+1)*episode_length]
        except:
            data = df.iloc[idx*episode_length:(idx+1)*remained_steps]

        first_row = data.iloc[0,:]
        obs = np.array([first_row['replica'], first_row['cpu'], first_row['heap'], first_row["cpu_usage"],first_row["memory_usage"],first_row["response_time"], first_row["expected_tps"]], dtype=np.float32)
        info = {}
        possible_state_value = np.array([first_row['replica'], first_row['cpu'], first_row['heap']])
        equal_rows = np.all(POSSIBLE_STATES == possible_state_value, axis=1)
        prev_action = np.where(equal_rows)[0][0]
        prev_reward = 0
        terminated = truncated = False
        # print("aaa",data.shape[0])
        for i in range(0, data.shape[0]):
            truncated = True if i == data.shape[0]-1 else False
            terminated = truncated
            selected_row = data.iloc[i,:]
            possible_state_value = np.array([selected_row['replica'], selected_row['cpu'], selected_row['heap']])
            equal_rows = np.all(POSSIBLE_STATES == possible_state_value, axis=1)
            action = np.where(equal_rows)[0][0]
            new_obs = np.array([selected_row['replica'], selected_row['cpu'], selected_row['heap'], selected_row["cpu_usage"],selected_row["memory_usage"],selected_row["response_time"], selected_row["expected_tps"]], dtype=np.float32)
            rew = selected_row["reward"]
            obs = obs/np.array([3,9,9,0.899,1.094, 835.78, 168])
            new_obs = new_obs/np.array([3,9,9,0.899,1.094, 835.78, 168])

            batch_builder.add_values(
                t=i,
                eps_id=eps_id,
                agent_index=0,
                # obs=prep.transform(obs),
                obs = obs,
                actions=action,
                action_prob=1.0, 
                action_logp=0.0,
                rewards=rew,
                prev_actions=prev_action,
                prev_rewards=prev_reward,
                terminateds=terminated,
                truncateds=truncated,
                infos=info,
                # new_obs=prep.transform(new_obs),
                new_obs = new_obs,

            )
            obs = new_obs
            prev_action = action
            prev_reward = rew
        writer.write(batch_builder.build_and_reset())


if __name__ == "__main__":
    batch_builder = SampleBatchBuilder()
    training_writer = JsonWriter(
        os.path.join(ray._private.utils.get_user_temp_dir(), "training-out")
    )
    eval_writer = JsonWriter(
        os.path.join(ray._private.utils.get_user_temp_dir(), "eval-out")
    )
    
    action_space = Discrete(108) #6*6*6
    observation_space = Box(low=np.array([1, 4, 4, 0,0,0,0]), high=np.array([3, 9, 9, 2,2, 1000,1000]), dtype=np.float32)
    replica = [1, 2, 3]
    cpu = [4, 5, 6, 7, 8, 9]
    heap = [4, 5, 6, 7, 8, 9]
    POSSIBLE_STATES = np.array(list(product(replica, cpu, heap)))


    prep = get_preprocessor(observation_space)(observation_space)
    print("The preprocessor is", prep)
    episode_length = 100
    full_data = hybrid_data
    training_split_ratio = 0.8
    train_df = full_data.sample(frac=training_split_ratio, random_state=42)  # Eğitim verisi
    eval_df = full_data.drop(train_df.index) 
    train_df = train_df.reset_index(drop=True)
    eval_df = eval_df.reset_index(drop=True)

    number_episodes_training = int(train_df.shape[0]/episode_length)
    remained_steps_training = train_df.shape[0]-number_episodes_training*episode_length
    if remained_steps_training > 0:
        number_episodes_training += 1

    # episode_length_eval = eval_df.shape[0]
    number_episodes_eval= int(eval_df.shape[0]/episode_length)
    remained_steps_eval = eval_df.shape[0]-number_episodes_eval*episode_length
    if remained_steps_eval > 0:
        number_episodes_eval += 1
        
    eps_id_training = list(range(number_episodes_training))
    eps_id_eval = list(range(len(eps_id_training), number_episodes_eval+ len(eps_id_training)))
    convert_data_to_batch(train_df, training_writer, eps_id_training)
    convert_data_to_batch(eval_df, eval_writer, eps_id_eval)


In [ ]:
from ray.rllib.algorithms.dqn import DQNConfig
from ray.rllib.offline.estimators import ImportanceSampling
from ray.rllib.offline.estimators.fqe_torch_model import FQETorchModel
from gymnasium.spaces import Discrete, Box
import numpy as np
from ray.rllib.offline.estimators import ImportanceSampling, WeightedImportanceSampling
from itertools import product
import matplotlib.pyplot as plt
import inspect


def generate_config(train_path, eval_path, hyper_params):
    learning_rate = hyper_params[0]
    fcnet_hiddens = hyper_params[1]
    config = (
        DQNConfig()
        .environment(env=None,
                    action_space=Discrete(108), 
                    observation_space=Box(low=np.array([1, 4, 4, 0,0,0,0]), high=np.array([3, 9, 9, 2,2, 1000,1000]), dtype=np.float32)
                    )
        .training(model={"fcnet_hiddens": fcnet_hiddens},
                  gamma=0,
                  lr=learning_rate,
                  train_batch_size=256
        )
                
        .offline_data(input_=train_path)
        .exploration(explore=False)
        .evaluation(evaluation_parallel_to_training=False,
                    evaluation_interval=2,
                    evaluation_duration=10,
                    evaluation_duration_unit="episodes",
                    evaluation_config={"input": eval_path},
                    off_policy_estimation_methods={"is": {"type": ImportanceSampling},
                                                   "wis": {"type": WeightedImportanceSampling}
                    }
        )
                    

    )
    return config



def generate_plots(filename,  mean_q_list, is_v_gain_list,wis_v_gain_list):
    fig, ax = plt.subplots(nrows=len(inspect.signature(generate_plots).parameters)-1,ncols=1,figsize=(12,36))
    ax[0].plot(mean_q_list)
    ax[0].set_xlabel('step')
    ax[0].set_ylabel('mean_q')
    ax[1].plot(is_v_gain_list)
    ax[1].set_xlabel('step')
    ax[1].set_ylabel('is_v_gain')
    ax[2].plot(wis_v_gain_list)
    ax[2].set_xlabel('step')
    ax[2].set_ylabel('wis_v_gain')
    plt.savefig(filename)


hyperparameters = {"learning_rate": [1e-05],
                   "fcnet_hiddens": [[64,64]]
                   }
parameter_combinations = list(product(*hyperparameters.values())) # This variable includes all combinations of the hyperparameters. ex. (1e-05, [32, 32])


train_path = "/tmp/training-out"
eval_path = "/tmp/eval-out"
epoch_number = 10000

for comb in parameter_combinations:
    config = generate_config(train_path, eval_path, comb)
    print(f"Started training with lr: {comb[0]} and fcnet: {comb[1]}")
    mean_q_list = []
    is_v_gain_list = []
    wis_v_gain_list = []

    # config['observation_filter'] = "MeanStdFilter"
    algo = config.build()
    debug_dir = "{}checkpoints/".format(algo.logdir)
    filename = f"./server_client_v2_offline/results/{algo.logdir.split('/')[-2]}_lr_{comb[0]}_fcnet_{comb[1][0]}_{comb[1][1]}.pdf"
    for i in range(epoch_number):
        print("------------- Iteration", i+1, "-------------")
        results = algo.train()
        print("timesteps_total:", results['timesteps_total'])
        print("training_iteration_time_ms:", results['timers']['training_iteration_time_ms'])
        if 'evaluation' in results.keys():
            print("== Evaluation ==")
            if 'off_policy_estimator' in results['evaluation'].keys():
                print(results['evaluation']['off_policy_estimator'])
                is_v_gain_list.append(results['evaluation']['off_policy_estimator']["is"]["v_gain"])
                wis_v_gain_list.append(results['evaluation']['off_policy_estimator']["wis"]["v_gain"])


        if (i+1) % 5000 == 0:
            ma_checkpoint_dir = algo.save(checkpoint_dir=debug_dir)
            print(
                "An Algorithm checkpoint has been created inside directory: "
                f"'{ma_checkpoint_dir}'"
            )
        mean_q_list.append(results["info"]["learner"]["default_policy"]["learner_stats"]["mean_q"])

    algo.stop()
    generate_plots(filename, mean_q_list,is_v_gain_list,wis_v_gain_list)


In [1]:
# Testing

from gevent import monkey
monkey.patch_all(thread=False, select=False)
from locust.env import Environment
from kubernetes import client, config

from prometheus_api_client import PrometheusConnect

from ray.rllib.env.policy_client import PolicyClient
import pandas as pd
import time
import numpy as np
from collections import OrderedDict
from gymnasium.spaces import Discrete, Dict, MultiDiscrete, Tuple, Box
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.algorithms.dqn import DQNConfig

import ssl
import random
import logging
import ray
from ray.rllib.algorithms.algorithm import Algorithm
from ray.rllib.env.policy_server_input import PolicyServerInput
from locust import HttpUser, task, constant, constant_throughput, events
from locust.shape import LoadTestShape

ssl._create_default_https_context = ssl._create_unverified_context
from itertools import product
import time


DEPLOYMENT_NAME = "teastore-webui"
NAMESPACE = "app2scale-test"

OBSERVATION_SPACE =Box(low=np.array([1, 4, 4, 0,0,0,0]), high=np.array([3, 9, 9, 2,2, 1000,1000]), dtype=np.float32)

"""
    # replica : 1,2,3,4,5,6 -> 0,1,2,3,4,5 + 1
    # cpu : 4,5,6,7,8,9 -> 0,1,2,3,4,5   +   4
    # heap : 4,5,6,7,8,9 -> 0,1,2,3,4,5   +   4
"""

ACTION_SPACE = Discrete(108) # index of the possible states
replica = [1, 2, 3]
cpu = [4, 5, 6, 7, 8, 9]
heap = [4, 5, 6, 7, 8, 9]

config_dqn = (DQNConfig()
          .environment(
              env=None,
              action_space=ACTION_SPACE,
              observation_space=OBSERVATION_SPACE)

          .training(model={"fcnet_hiddens": [64,64]},
              gamma=0.99,
              lr=1e-05,
              train_batch_size=256)

          .debugging(log_level="INFO")
          .evaluation(off_policy_estimation_methods={})
          
          )


config_dqn.rl_module(_enable_rl_module_api=False)
config_dqn.training(_enable_learner_api=False)
algo = config_dqn.build() 


path_to_checkpoint = "/Users/hasan.nayir/ray_results/DQN_None_2024-02-27_22-50-28ooarimlb/checkpoints/checkpoint_010000"
algo.restore(path_to_checkpoint)

/Users/hasan.nayir/miniconda3/envs/rl_lib/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/hasan.nayir/miniconda3/envs/rl_lib/lib/python3.9/site-packages/gymnasium/spaces/box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")
2024-02-28 00:00:33,206	INFO policy.py:1285 -- Policy (worker=local) running on CPU.
2024-02-28 00:00:33,209	INFO torch_policy.py:184 -- Found 0 visible cuda devices.
2024-02-28 00:00:33,216	INFO util.py:118 -- Using connectors:
2024-02-28 00:00:33,217	INFO util.py:119 --     AgentConnectorPipeline
        ObsPreprocessorConnector
        StateBufferConnector
        ViewRequirementAgentConnector
2024-02-28 00:00:33,217	INFO util.py:120 --     ActionConnectorPipeline
        Conv

In [4]:
OBSERVATION_SPACE.sample()

array([  2.4114044,   5.6831293,   6.913543 ,   1.9389706,   1.8907213,
       506.15012  , 560.51086  ], dtype=float32)

obs = np.array([first_row['replica'], first_row['cpu'], first_row['heap'], first_row["cpu_usage"],first_row["memory_usage"],first_row["response_time"], first_row["expected_tps"]], dtype=np.float32)


In [43]:
ACTION_SPACE = Discrete(108) # index of the possible states
replica = [1, 2, 3]
cpu = [4, 5, 6, 7, 8, 9]
heap = [4, 5, 6, 7, 8, 9]

POSSIBLE_STATES = np.array(list(product(replica, cpu, heap)))


test_step = 100
for i in range(test_step):
    obs = OBSERVATION_SPACE.sample()/np.array([3,9,9,0.899,1.094, 835.78, 168])
    action = algo.compute_single_action(obs)
    print(action)
    print(POSSIBLE_STATES[action])


92
[3 7 6]
60
[2 8 4]
42
[2 5 4]
85
[3 6 5]
42
[2 5 4]
0
[1 4 4]
42
[2 5 4]
92
[3 7 6]
92
[3 7 6]
42
[2 5 4]
93
[3 7 7]
92
[3 7 6]
0
[1 4 4]
60
[2 8 4]
85
[3 6 5]
85
[3 6 5]
85
[3 6 5]
26
[1 8 6]
26
[1 8 6]
26
[1 8 6]
93
[3 7 7]
92
[3 7 6]
60
[2 8 4]
92
[3 7 6]
42
[2 5 4]
69
[2 9 7]
42
[2 5 4]
60
[2 8 4]
7
[1 5 5]
85
[3 6 5]
85
[3 6 5]
42
[2 5 4]
60
[2 8 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
93
[3 7 7]
42
[2 5 4]
92
[3 7 6]
42
[2 5 4]
92
[3 7 6]
42
[2 5 4]
42
[2 5 4]
92
[3 7 6]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
60
[2 8 4]
42
[2 5 4]
42
[2 5 4]
85
[3 6 5]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
92
[3 7 6]
92
[3 7 6]
42
[2 5 4]
60
[2 8 4]
85
[3 6 5]
26
[1 8 6]
60
[2 8 4]
92
[3 7 6]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
60
[2 8 4]
0
[1 4 4]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
85
[3 6 5]
42
[2 5 4]
92
[3 7 6]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
69
[2 9 7]
69
[2 9 7]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
60
[2 8 4]
85
[3 6 5]
42
[2 5 4]
42
[2 5 4]
42
[2 5 4]
92
[3 7 6]
42
